# Efficient Frontier Portfolio Optimization

## Comprehensive Analysis with Modern Portfolio Theory

This notebook demonstrates a complete efficient frontier portfolio optimization system with strict performance constraints:
- **Minimum Annualized Return**: 20%
- **Minimum Sharpe Ratio**: 2.0
- **Maximum Drawdown**: 3%

### Table of Contents
1. [Setup and Data Collection](#setup)
2. [Data Analysis and Validation](#data-analysis)
3. [Modern Portfolio Theory Implementation](#mpt)
4. [Constraint Optimization](#constraints)
5. [Efficient Frontier Generation](#frontier)
6. [Results Analysis and Visualization](#results)
7. [Portfolio Composition and Risk Analysis](#composition)
8. [Backtesting and Performance Evaluation](#backtesting)
9. [Conclusions and Recommendations](#conclusions)

## 1. Setup and Data Collection {#setup}

First, let's import all necessary libraries and set up our analysis environment.

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Optimization libraries
import cvxpy as cp
from scipy.optimize import minimize
from scipy import stats

# Data fetching
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Custom modules
from data_fetcher import MarketDataFetcher
from portfolio_optimizer import EfficientFrontierOptimizer, OptimizationConstraints
from portfolio_metrics import PortfolioMetrics
from visualization import PortfolioVisualizer
from export_results import ResultsExporter

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

### Configuration Parameters

Define the key parameters for our optimization analysis:

In [ ]:
# Analysis Configuration
CONFIG = {
    'num_assets': 100,           # Number of top assets to analyze
    'period_years': 5,           # Years of historical data
    'min_return': 0.20,          # Minimum 20% annualized return
    'min_sharpe': 2.0,           # Minimum Sharpe ratio of 2.0
    'max_drawdown': 0.03,        # Maximum 3% drawdown
    'risk_free_rate': 0.02,      # 2% risk-free rate
    'n_frontier_portfolios': 100, # Number of efficient frontier points
}

print("📋 Analysis Configuration:")
print(f"   Target Assets: {CONFIG['num_assets']}")
print(f"   Historical Period: {CONFIG['period_years']} years")
print(f"   Min Return Constraint: {CONFIG['min_return']:.1%}")
print(f"   Min Sharpe Constraint: {CONFIG['min_sharpe']:.1f}")
print(f"   Max Drawdown Constraint: {CONFIG['max_drawdown']:.1%}")
print(f"   Risk-Free Rate: {CONFIG['risk_free_rate']:.1%}")

### Data Collection

Now let's fetch the top assets by trading volume and download their historical data:

In [ ]:
# Initialize data fetcher
print("🔄 Initializing data fetcher...")
data_fetcher = MarketDataFetcher(period_years=CONFIG['period_years'])

# Get top assets by trading volume
print(f"📊 Identifying top {CONFIG['num_assets']} assets by trading volume...")
symbols = data_fetcher.get_top_assets_by_volume(num_assets=CONFIG['num_assets'])

print(f"\n🎯 Selected {len(symbols)} assets:")
print(f"First 20: {symbols[:20]}")
if len(symbols) > 20:
    print(f"... and {len(symbols)-20} more")

In [ ]:
# Fetch historical price data
print(f"📈 Downloading {CONFIG['period_years']} years of historical data...")
prices = data_fetcher.fetch_historical_data(symbols)
returns = data_fetcher.calculate_returns()

print(f"\n✅ Data collection completed:")
print(f"   Assets: {len(data_fetcher.symbols)}")
print(f"   Date Range: {prices.index[0].date()} to {prices.index[-1].date()}")
print(f"   Trading Days: {len(returns)}")
print(f"   Data Shape: {returns.shape}")

## 2. Data Analysis and Validation {#data-analysis}

Let's analyze the quality and characteristics of our data:

In [ ]:
# Data validation
validation = data_fetcher.validate_data_quality()
data_summary = data_fetcher.get_data_summary()

print("🔍 Data Quality Assessment:")
print(f"   Valid: {validation['valid']}")
print(f"   Data Completeness: {data_summary['data_completeness']:.2%}")

if validation.get('warnings'):
    print("\n⚠️  Warnings:")
    for warning in validation['warnings']:
        print(f"   - {warning}")

# Display basic statistics
print("\n📊 Returns Statistics:")
returns_stats = returns.describe()
print(returns_stats.round(4))

## 3. Modern Portfolio Theory Implementation {#mpt}

Now let's implement the core MPT optimization engine:

In [ ]:
# Initialize the optimizer
print("🔧 Initializing Portfolio Optimizer...")
optimizer = EfficientFrontierOptimizer(
    returns=returns,
    risk_free_rate=CONFIG['risk_free_rate']
)

# Display expected returns and covariance matrix properties
print(f"\n📊 Portfolio Universe Properties:")
print(f"   Number of Assets: {optimizer.n_assets}")
print(f"   Expected Returns Range: {optimizer.expected_returns.min():.2%} to {optimizer.expected_returns.max():.2%}")
print(f"   Covariance Matrix Condition Number: {np.linalg.cond(optimizer.cov_matrix.values):.0f}")

# Check if covariance matrix is well-conditioned
eigenvals = np.linalg.eigvals(optimizer.cov_matrix.values)
print(f"   Covariance Matrix Eigenvalues: min={eigenvals.min():.6f}, max={eigenvals.max():.6f}")

if np.any(eigenvals <= 0):
    print("   ⚠️  Warning: Covariance matrix has non-positive eigenvalues")
else:
    print("   ✅ Covariance matrix is positive definite")

## 4. Constraint Optimization {#constraints}

Let's set up and test our optimization constraints:

In [ ]:
# Define optimization constraints
constraints = OptimizationConstraints(
    min_return=CONFIG['min_return'],
    min_sharpe=CONFIG['min_sharpe'],
    max_drawdown=CONFIG['max_drawdown'],
    min_weight=0.0,
    max_weight=0.20,  # Maximum 20% in any single asset
    max_assets=50     # Maximum 50 assets in portfolio
)

print("🎯 Optimization Constraints:")
print(f"   Minimum Return: {constraints.min_return:.1%}")
print(f"   Minimum Sharpe Ratio: {constraints.min_sharpe:.1f}")
print(f"   Maximum Drawdown: {constraints.max_drawdown:.1%}")
print(f"   Weight Range: {constraints.min_weight:.1%} - {constraints.max_weight:.1%}")
print(f"   Maximum Assets: {constraints.max_assets}")

## 5. Efficient Frontier Generation {#frontier}

Generate the complete efficient frontier and find optimal portfolios:

In [ ]:
# Find optimal portfolios
print("🎯 Finding Optimal Portfolios...")
optimal_portfolios = optimizer.find_optimal_portfolios(constraints)

print(f"\n📊 Optimal Portfolio Results:")
for name, portfolio in optimal_portfolios.items():
    if portfolio['weights'] is not None:
        metrics = portfolio['metrics']
        constraints_met = portfolio['constraints_met']
        
        print(f"\n   {name.upper().replace('_', ' ')}:")
        print(f"     Return: {metrics['annualized_return']:.2%}")
        print(f"     Volatility: {metrics['annualized_volatility']:.2%}")
        print(f"     Sharpe Ratio: {metrics['sharpe_ratio']:.3f}")
        print(f"     Max Drawdown: {metrics['max_drawdown']:.2%}")
        print(f"     All Constraints Met: {'✅' if constraints_met['all_constraints'] else '❌'}")
    else:
        print(f"\n   {name.upper().replace('_', ' ')}: ❌ Optimization failed")

## 6. Results Analysis and Visualization {#results}

Create comprehensive visualizations of our optimization results:

In [ ]:
# Generate efficient frontier
print(f"\n🌐 Generating Efficient Frontier ({CONFIG['n_frontier_portfolios']} portfolios)...")
frontier_df = optimizer.generate_efficient_frontier(
    n_portfolios=CONFIG['n_frontier_portfolios'],
    constraints=constraints
)

# Initialize visualizer and create plots
visualizer = PortfolioVisualizer()

if not frontier_df.empty:
    print("📊 Creating Efficient Frontier Visualization...")
    visualizer.plot_efficient_frontier(
        frontier_df, 
        optimal_portfolios,
        interactive=True
    )
    
    # Performance comparison
    if optimal_portfolios:
        print("📊 Creating Performance Comparison...")
        visualizer.plot_performance_comparison(optimal_portfolios)
else:
    print("⚠️  Cannot create plots - no frontier data available")

## 7. Portfolio Composition Analysis {#composition}

Analyze the composition of our best portfolio:

In [ ]:
# Find best portfolio
best_portfolio = None
best_score = -np.inf

for portfolio in optimal_portfolios.values():
    if portfolio['weights'] is not None:
        constraints_met = portfolio['constraints_met']['all_constraints']
        sharpe_ratio = portfolio['metrics']['sharpe_ratio']
        score = (10 if constraints_met else 0) + sharpe_ratio
        
        if score > best_score:
            best_score = score
            best_portfolio = portfolio

if best_portfolio:
    print("🏆 Best Portfolio Analysis:")
    composition = optimizer.get_portfolio_composition(best_portfolio['weights'], top_n=15)
    print(composition)
    
    # Visualize composition
    visualizer.plot_portfolio_composition(
        best_portfolio['weights'],
        data_fetcher.symbols,
        title="Optimal Portfolio Composition"
    )
else:
    print("❌ No optimal portfolio found")

## 8. Backtesting and Performance Evaluation {#backtesting}

Evaluate the historical performance of our optimal portfolio:

In [ ]:
if best_portfolio:
    # Run backtest
    print("📈 Running Backtest Analysis...")
    backtest_results = optimizer.backtest_portfolio(best_portfolio['weights'])
    
    # Display results
    print(f"\n📊 Backtest Results:")
    print(f"   Period: {backtest_results['start_date'].date()} to {backtest_results['end_date'].date()}")
    print(f"   Total Return: {(backtest_results['cumulative_returns'].iloc[-1] - 1):.2%}")
    print(f"   Annualized Return: {backtest_results['metrics']['annualized_return']:.2%}")
    print(f"   Annualized Volatility: {backtest_results['metrics']['annualized_volatility']:.2%}")
    print(f"   Sharpe Ratio: {backtest_results['metrics']['sharpe_ratio']:.3f}")
    print(f"   Maximum Drawdown: {backtest_results['metrics']['max_drawdown']:.2%}")
    
    # Plot cumulative returns
    plt.figure(figsize=(12, 6))
    plt.plot(backtest_results['cumulative_returns'].index, 
             backtest_results['cumulative_returns'].values, 
             linewidth=2, label='Optimal Portfolio')
    
    # Compare with equal-weighted benchmark
    benchmark_returns = returns.mean(axis=1)
    benchmark_cumulative = (1 + benchmark_returns).cumprod()
    plt.plot(benchmark_cumulative.index, benchmark_cumulative.values, 
             linewidth=2, alpha=0.7, label='Equal-Weighted Benchmark')
    
    plt.title('Portfolio Performance Comparison')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("❌ Cannot run backtest - no optimal portfolio available")

## 9. Conclusions and Recommendations {#conclusions}

### Key Findings

Based on our comprehensive analysis, here are the main findings:

1. **Constraint Feasibility**: The constraints (20% return, 2.0 Sharpe, 3% max drawdown) are extremely strict for most market conditions.

2. **Portfolio Optimization**: Modern Portfolio Theory provides a robust framework for optimization, but real-world constraints may limit feasible solutions.

3. **Risk-Return Tradeoff**: Higher returns typically require accepting higher risk, making the combination of high return and low drawdown challenging.

### Recommendations

1. **Constraint Relaxation**: Consider more realistic constraints:
   - Minimum return: 12-15%
   - Minimum Sharpe ratio: 1.0-1.5
   - Maximum drawdown: 5-10%

2. **Diversification**: Expand the asset universe to include:
   - International markets
   - Alternative assets (REITs, commodities)
   - Fixed income securities

3. **Dynamic Rebalancing**: Implement regular portfolio rebalancing to maintain optimal weights.

4. **Risk Management**: Use additional risk controls such as:
   - Stop-loss mechanisms
   - Volatility targeting
   - Regime-aware optimization

### Next Steps

1. **Implementation**: If feasible portfolios were found, consider implementing with proper risk management.

2. **Monitoring**: Regularly monitor portfolio performance and rebalance as needed.

3. **Enhancement**: Consider advanced techniques like:
   - Black-Litterman model
   - Risk parity approaches
   - Machine learning-based optimization

4. **Stress Testing**: Perform stress tests under various market scenarios.

## Export Results

Finally, let's export our results for further analysis:

In [ ]:
# Export results
exporter = ResultsExporter()

if not frontier_df.empty:
    print("💾 Exporting results...")
    
    # Export efficient frontier
    frontier_file = exporter.export_efficient_frontier(frontier_df)
    
    # Export optimal portfolios
    if optimal_portfolios:
        portfolios_file = exporter.export_optimal_portfolios(
            optimal_portfolios, data_fetcher.symbols
        )
    
    # Create summary report
    results_dict = {
        'data_summary': data_fetcher.get_data_summary(),
        'optimal_portfolios': optimal_portfolios,
        'frontier_df': frontier_df,
        'symbols': data_fetcher.symbols
    }
    
    summary_file = exporter.create_summary_report(results_dict)
    
    print("✅ All results exported successfully!")
else:
    print("⚠️  No results to export")

print("\n🎉 Analysis Complete!")
print("Check the results/ directory for exported files.")